<a href="https://colab.research.google.com/github/un21988/CE888-7-project-2/blob/main/Emotion_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [4]:
!pip install transformers

     |████████████████████████████████| 1.8MB 7.8MB/s 
     |████████████████████████████████| 3.2MB 28.4MB/s 
     |████████████████████████████████| 890kB 46.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=179f8a7c5649b405f7d8740b546c0a415f7b7b087d2bb4d8a6ce738ca544811a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [8]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [9]:
# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]
labels

['anger', 'joy', 'optimism', 'sadness']

In [10]:
import pandas as pd
import urllib.request 
import requests

test_text_url = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_text.txt"
val_text_url= "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/val_text.txt"
val_lable_url= "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/val_labels.txt"
#df = open(url,"r")
#print(df)


r = requests.get(test_text_url, allow_redirects=True)
open('test_text.txt', 'wb').write(r.content)

r1 = requests.get(val_text_url, allow_redirects=True)
open('val_text.txt', 'wb').write(r1.content)


r2 = requests.get(val_lable_url, allow_redirects=True)
open('val_lable.txt', 'wb').write(r2.content)






748

In [11]:

stream=open("test_text.txt")
tweets=stream.readlines()
stream.close()
print(tweets[1])

@user Interesting choice of words... Are you confirming that governments fund #terrorism? Bit of an open door, but still... 



In [12]:
stream1=open("val_lable.txt")
val_lable=stream1.readlines()
stream1.close()
print(val_lable)

['0\n', '0\n', '0\n', '0\n', '0\n', '0\n', '3\n', '3\n', '0\n', '3\n', '1\n', '0\n', '0\n', '3\n', '3\n', '0\n', '0\n', '3\n', '3\n', '0\n', '3\n', '1\n', '2\n', '3\n', '3\n', '3\n', '0\n', '1\n', '0\n', '0\n', '0\n', '3\n', '0\n', '3\n', '2\n', '1\n', '1\n', '0\n', '2\n', '1\n', '1\n', '3\n', '1\n', '0\n', '1\n', '1\n', '3\n', '0\n', '0\n', '0\n', '2\n', '1\n', '1\n', '3\n', '0\n', '0\n', '0\n', '0\n', '1\n', '1\n', '3\n', '2\n', '3\n', '1\n', '3\n', '0\n', '0\n', '0\n', '3\n', '0\n', '0\n', '0\n', '1\n', '2\n', '1\n', '0\n', '1\n', '0\n', '1\n', '0\n', '3\n', '0\n', '0\n', '1\n', '0\n', '0\n', '3\n', '0\n', '3\n', '3\n', '0\n', '1\n', '3\n', '1\n', '0\n', '1\n', '3\n', '2\n', '0\n', '2\n', '2\n', '1\n', '1\n', '1\n', '3\n', '3\n', '0\n', '0\n', '1\n', '3\n', '1\n', '0\n', '1\n', '1\n', '3\n', '1\n', '1\n', '0\n', '1\n', '1\n', '0\n', '0\n', '0\n', '0\n', '0\n', '3\n', '1\n', '0\n', '0\n', '3\n', '0\n', '1\n', '0\n', '0\n', '0\n', '2\n', '0\n', '3\n', '3\n', '0\n', '1\n', '0\n', '0\n'

In [13]:
stream2=open("val_text.txt")
val_text=stream2.readlines()
stream2.close()
print(val_text[1])

if not then #teamchristine bc all tana has done is provoke her by tweeting shady shit and trying to be a hard bitch begging for a fight 



In [14]:
tweet_preprocess=[]

for i in range(0,len(tweets)):
  print(tweets[i])
 # tweets.append(preprocess(tweets[i]))
  tweet_preprocess.append(preprocess(tweets[i]))
  if i==10:
    break

print(tweet_preprocess[0])

#Deppression is real. Partners w/ #depressed people truly dont understand the depth in which they affect us. Add in #anxiety &amp;makes it worse 

@user Interesting choice of words... Are you confirming that governments fund #terrorism? Bit of an open door, but still... 

My visit to hospital for care triggered #trauma from accident 20+yrs ago and image of my dead brother in it. Feeling symptoms of #depression 

@user Welcome to #MPSVT! We are delighted to have you! #grateful #MPSVT #relationships 

What makes you feel #joyful? 

i am revolting. 

Rin might ever appeared gloomy but to be a melodramatic person was not her thing.\n\nBut honestly, she missed her old friend. The special one. 

In need of a change! #restless 

@user @user #cmbyn does screen  August 4 &amp; 6 at #miff 

@user Get Donovan out of your soccer booth. He's awful. He's bitter. He makes me want to mute the tv. #horrid 

@user how can u have sold so many copies but ur game has so many fucking bugs and mad lag issues

In [15]:
result=[]
for i in  range(0,len(tweet_preprocess)):
  result.append(model(tweet_preprocess[i]))
  if i==2:
    break
 
#print(result)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model ch

In [17]:
print(result)


[['1) sadness 0.966', '2) anger 0.0184', '3) optimism 0.011', '4) joy 0.0046'], ['1) optimism 0.4538', '2) anger 0.2607', '3) sadness 0.2034', '4) joy 0.0822'], ['1) sadness 0.984', '2) anger 0.007', '3) joy 0.005', '4) optimism 0.004']]


In [18]:

def model(tweets):

 model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)

 text = tweets
 encoded_input = tokenizer(text, return_tensors='tf')
 output = model(encoded_input)
 scores = output[0][0].numpy()
 scores = softmax(scores)

 temp_list=[]
 ranking = np.argsort(scores)
 ranking = ranking[::-1]
 for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    temp_list.append(f"{i+1}) {l} {np.round(float(s), 4)}")
 return temp_list